### Generate stimulus:

In [16]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3

nActsX = 5
nActsY = 5
nWeights = 3

a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActsX,nActsY))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

In [17]:
a

array([[-3, -2, -2,  1, -3],
       [-2, -1, -1, -4, -2],
       [ 0, -3, -2, -3, -4],
       [ 2, -2,  2,  1, -4],
       [-3, -1, -2,  0,  1]])

In [18]:
w

array([[-2, -2, -3],
       [ 0,  2, -2],
       [-2,  1, -2]])

In [19]:
o

array([[17, 21, 16],
       [-3, 22, 23],
       [13, 21, 34]])

### Prepare scan chain for multicast ID

In [20]:
ids = w.copy()
for x,idx in enumerate(ids):
    for y,id in enumerate(idx):
        ids[x][y] = x + y
ids.flatten()

array([0, 1, 2, 1, 2, 3, 2, 3, 4])

### Save stimulus:

In [ ]:
path = '/home/lquizon/lawrence-workspace/eyeriss/tb/cluster'
# np.savetxt(path + '/a.txt',a,'%i')
# np.savetxt(path + '/w.txt',w,'%i')
# np.savetxt(path + '/o.txt',o,'%i')

### Have a look at existing stimulus:

In [22]:
import numpy as np
path = '.'
a = np.loadtxt(path + '/a.txt')
w = np.loadtxt(path + '/w.txt')
o= np.loadtxt(path+'/o.txt')  

### Row-stationary Forensics

In [65]:
osize = nActsX-nWeights+1
sequence_array = [[
    (0, 0), # 0,0
    (1, 1), # 
    (2, 2),
    ],[
    (0, 1),
    (1, 2),
    (2, 3),
    ],[
    (0, 2),
    (1, 3),
    (2, 4)
    ]
]
ps = []
psr = []

for sequence in sequence_array:
    psr = []
    for wrow,arow in sequence:
        psr.append(np.convolve(a[arow],w[wrow][::-1],'valid'))   
    ps.append(psr)

np.array(ps)

# Expected PE partial sums:

array([[[ 16.,   5.,  11.],
        [  0.,   6.,  -4.],
        [  1.,  10.,   9.]],

       [[  9.,  16.,  16.],
        [ -2.,   2.,   2.],
        [-10.,   4.,   5.]],

       [[ 12.,  19.,  22.],
        [ -8.,   2.,  10.],
        [  9.,   0.,   2.]]])

In [64]:
ress = []
for psr in ps:
    res = psr.copy()
    for i in range(1,len(res)):
        res[i] += res[i-1]
    ress.append(res)
np.array(ress)

# Expected partial sum outputs in systolic passing:

array([[[16.,  5., 11.],
        [16., 11.,  7.],
        [17., 21., 16.]],

       [[ 9., 16., 16.],
        [ 7., 18., 18.],
        [-3., 22., 23.]],

       [[12., 19., 22.],
        [ 4., 21., 32.],
        [13., 21., 34.]]])